In [1]:
# Copyright 2018 Anar Amirli
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train/Test Set Construction
In this document, we construct training and test sets from already computed feature sets. The sets are computed according to the layout:

data<br>
├── train_features<br>
├── test_features<br>

In [2]:
from os import listdir
from os.path import join
import pickle
import numpy as np
import pandas as pd
import multiprocessing
from collections import Counter
from random import shuffle
import json
import re

# our common utility functions that are used in different notebooks
from utils import *

pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

In [3]:
def describe_dataset(dataset):
    print('Shape: {}\n'.format(dataset.shape))
    
    _, col = dataset.shape
        
    heading = 'Pitch index counts'
    print(heading + '\n' + '-'*len(heading))
    for key, val in sorted(Counter(dataset.values[:, col-1]).items()):
        print('{}\t: {}'.format(int(key), val))

## Pitch IDS

In [4]:
if __name__ == '__main__':

    match_id = 60565
    try:
        features_df = pd.read_csv('../data/general/feature-set/match{}_features_2.csv'.format(match_id))
    except FileNotFoundError:
        pass

    ball_idxs = np.sort(features_df['pitch_index'].unique())
    print('Ball  index: {}'.format(ball_idxs))

    print('Size: {}'.format(len(ball_idxs)))

Ball  index: [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.]
Size: 16


Null index(0) represents all the event categories, apart from the ones we are interested in, that occur when the game stops.

## Dataset Construction

In this section, we construct a combined shuffled dataset for train and set sets, coming from feature data.


### Parameters

When obtaining feature data for a given index, set we should reduce the number of frames of the null ball events in our data set. But cosidering the fact that they are most common, we should keep their numbers close to the number of other most occurred ball location indexes.

* $-$: No-labaled positions intervals would lead to greater label noise.

### Construction
Here we construct the combined dataset from all match data we have in the given data directories.

In [5]:
    # file based constants
    features_dir     = '../data/general/feature-set/'
    features_regex   = re.compile(r'match\d+_features_2.csv')

We speed up the computation by using all the CPU cores via multiprocessing module.

In [6]:
    pool = multiprocessing.Pool()
    df = pd.DataFrame()

    
    features_csv_files = [f for f in listdir(features_dir) if features_regex.match(f)]
    shuffle(features_csv_files)
    print('All matches: {}'.format(len(features_csv_files)))
    features_train_csv = features_csv_files[0:259]
    features_test_csv  = features_csv_files[259:len(features_csv_files)]
    print('Train matches: {}'.format(len(features_train_csv)))
    print('Test matches: {}'.format(len(features_test_csv)))
    
    
    df_train = pd.concat(pool.map(construct_test_set, features_train_csv))
    df_test = pd.concat(pool.map(construct_test_set, features_test_csv))


    print('Train-set shape: {}'.format(df_train.shape))
    print('Test-set shape: {}'.format(df_test.shape))

All matches: 298
Train matches: 259
Test matches: 39
Train-set shape: (586544, 228)
Test-set shape: (92838, 228)


In [7]:
    # shuffling data-sets agin
    df_train = df_train[df_train['pitch_index']!=0]
    df_test = df_test[df_test['pitch_index']!=0]
    
    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_test = df_test.sample(frac=1).reset_index(drop=True)

### Train Set

In [8]:
    describe_dataset(df_train)
    display(df_train.head())

Shape: (212671, 228)

Pitch index counts
------------------
1	: 4870
2	: 24010
3	: 4932
4	: 14192
5	: 17801
6	: 14232
7	: 21054
8	: 21560
9	: 21434
10	: 15982
11	: 12330
12	: 17590
13	: 7622
14	: 5952
15	: 9110


,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
0,53.62500,39.81250,1.419922,-1.0,1.0,47.8750,47.156250,42.78125,45.000000,63.87500,...,36.12500,1.820312,37.53125,40.81250,47.62500,31.453125,0.00,0.0000,0.000000,9.0
1,36.00000,43.87500,1.629883,1.0,1.0,49.5625,48.562500,38.56250,46.750000,59.68750,...,63.75000,1.790039,0.00000,0.00000,43.53125,63.750000,0.00,0.0000,0.000000,6.0
2,72.87500,33.18750,1.509766,-1.0,-1.0,78.1875,36.625000,64.12500,42.218750,84.00000,...,51.68750,4.390625,0.00000,0.00000,0.00000,0.000000,54.25,51.6875,4.390625,9.0
3,82.81250,13.34375,1.990234,-1.0,1.0,84.3125,25.578125,84.25000,26.296875,88.62500,...,32.34375,1.110352,51.87500,32.34375,0.00000,0.000000,0.00,0.0000,0.000000,12.0
4,54.28125,24.87500,1.099609,-1.0,-1.0,47.2500,25.234375,44.96875,27.671875,62.71875,...,23.06250,1.995117,0.00000,0.00000,38.28125,23.062500,0.00,0.0000,0.000000,7.0


### Test Set

In [9]:
    describe_dataset(df_test)
    display(df_test.head())

Shape: (32930, 228)

Pitch index counts
------------------
1	: 739
2	: 3781
3	: 789
4	: 2147
5	: 2858
6	: 2236
7	: 3103
8	: 3110
9	: 3347
10	: 2592
11	: 1878
12	: 2764
13	: 1238
14	: 919
15	: 1429


,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
0,55.187500,30.6250,3.669922,1.0,1.0,41.9375,32.78125,40.406250,33.9375,69.06250,...,39.81250,3.359375,0.0000,0.00000,0.00000,0.000,27.875,48.1875,3.449219,2.0
1,25.234375,47.4375,2.550781,1.0,1.0,14.0000,43.28125,11.765625,45.9375,40.78125,...,32.40625,0.930176,13.8750,32.40625,0.00000,0.000,0.000,0.0000,0.000000,3.0
2,73.062500,49.8750,1.219727,1.0,-1.0,81.0000,55.15625,75.500000,64.3125,89.37500,...,64.75000,1.480469,69.3125,64.75000,0.00000,0.000,0.000,0.0000,0.000000,12.0
3,61.937500,30.5625,1.299805,-1.0,1.0,62.5000,37.81250,57.468750,34.2500,67.93750,...,37.37500,2.785156,0.0000,0.00000,49.59375,37.375,0.000,0.0000,0.000000,9.0
4,85.000000,16.6875,2.199219,1.0,-1.0,90.1875,24.53125,78.562500,27.1250,95.75000,...,0.00000,0.000000,0.0000,0.00000,0.00000,0.000,0.000,0.0000,0.000000,13.0


## Exporting

In [10]:
    df_train.to_csv('../data/general/{dataset}/all_{dataset}_2.csv'.format(dataset='train'), index=False)
    df_test.to_csv('../data/general/{dataset}/all_{dataset}_2.csv'.format(dataset='test'), index=False)

In [11]:
    df_train[1:10]

,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
1,36.00000,43.87500,1.629883,1.0,1.0,49.56250,48.562500,38.56250,46.750000,59.68750,...,63.750000,1.790039,0.00000,0.000000,43.53125,63.75000,0.000,0.00000,0.000000,6.0
2,72.87500,33.18750,1.509766,-1.0,-1.0,78.18750,36.625000,64.12500,42.218750,84.00000,...,51.687500,4.390625,0.00000,0.000000,0.00000,0.00000,54.250,51.68750,4.390625,9.0
3,82.81250,13.34375,1.990234,-1.0,1.0,84.31250,25.578125,84.25000,26.296875,88.62500,...,32.343750,1.110352,51.87500,32.343750,0.00000,0.00000,0.000,0.00000,0.000000,12.0
4,54.28125,24.87500,1.099609,-1.0,-1.0,47.25000,25.234375,44.96875,27.671875,62.71875,...,23.062500,1.995117,0.00000,0.000000,38.28125,23.06250,0.000,0.00000,0.000000,7.0
5,74.25000,38.28125,1.730469,1.0,-1.0,71.93750,45.937500,60.50000,23.140625,83.50000,...,46.718750,2.140625,0.00000,0.000000,63.15625,46.71875,0.000,0.00000,0.000000,12.0
6,74.50000,37.12500,2.539062,-1.0,-1.0,85.56250,29.109375,73.75000,40.531250,94.37500,...,40.718750,1.735352,78.43750,41.156250,47.96875,40.28125,0.000,0.00000,0.000000,14.0
7,82.75000,40.34375,0.770020,1.0,1.0,87.25000,41.093750,85.50000,39.187500,92.68750,...,36.250000,0.990234,80.06250,34.562500,66.00000,37.96875,0.000,0.00000,0.000000,15.0
8,58.28125,31.12500,4.101562,-1.0,1.0,52.78125,41.593750,53.96875,39.437500,72.56250,...,47.156250,4.363281,0.00000,0.000000,46.40625,39.40625,41.875,54.90625,5.781250,8.0
9,68.62500,9.78125,0.239990,1.0,-1.0,65.50000,19.218750,61.87500,25.078125,77.81250,...,22.953125,1.190430,54.28125,22.953125,0.00000,0.00000,0.000,0.00000,0.000000,10.0
